# 获取诺奖得主的非合作者groupD
要求：与诺奖得主没有合作，但有获奖主题论文发表，且其他获奖主题发表为0，获得groupD的待选列表1

* groupD_1：在诺奖颁布前就已经发表了获奖主题论文
* groupD_2：在诺奖颁布后才开始发表获奖主题论文

In [1]:
import pandas as pd
import json
from tkinter import _flatten
import os
import Ipynb_importer
import Module2_get_collasInfo_name2id as Module2

In [3]:
dirPath_1 = 'D:/Anaconda/Code_and_Data/8 Nobel_Prize_Analyze/results-datasets/'
dirPath_3 = 'D:/Anaconda/Code_and_Data/8.1 Nobel_Prize_Analyze_apsNew/results-datasets/'

In [10]:
# 函数：获取给定作者的发文主题信息，如果其发文主题与待删除的主题有重合，则需要删除该作者
def del_author_underTopics_func(author_id, del_topic_list, allAuthors_to_topicInfo_dict):
    topic_dict_i = allAuthors_to_topicInfo_dict[author_id]
    topic_list_i = list(topic_dict_i.keys())
    
    if set(topic_list_i) & set(del_topic_list):
        return True
    else:
        return False

In [ ]:
# 函数：将字典写入JSON文件
def dict_to_json_func1(dir_path, file_name, dump_dict):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    with open(dir_path+"/"+file_name, 'w', encoding='utf-8-sig') as fw:
        json.dump(dump_dict, fw)
    print("Successfully dump to json file! <{0}>".format(file_name))

In [16]:
def get_groupD_tempList_func(resolution_param, year_diff=None, cluster_param='global_clusterNum'):
    
    path5 = dirPath_1 + "/3-laureateId-to-collasId-dictData/allPaper_laurId2collaId_dict_addDOI.json"
    with open(path5, 'r', encoding='utf-8-sig') as fr5:
        allLaurs_id2collas_allPapers_dict = json.load(fr5)
        
    path6 = dirPath_3 + "/1-paperAll-clustering-data-CCN/allAuthors_id2paperInfo_withGlobalCluster/resolutionParam_{0}.json".format(resolution_param)
    with open(path6, 'r', encoding='utf-8-sig') as fr6:
        allAuthors_id2paperInfo_dict = json.load(fr6)
    
    path_1 = dirPath_3 + '/3-groupA-matching/allLaurs_id2groupAInfo_{0}/resolutionParam_{1}.json'.format(cluster_param,str(resolution_param))
    with open(path_1, 'r', encoding='utf-8-sig') as fr_1:
        allLaurs_id2groupAInfo_dict = json.load(fr_1)
        
    # 所有诺奖得主的ID列表
    all_laursID_list = list(allLaurs_id2groupAInfo_dict.keys())
    path_2 = dirPath_3 + "/1-paperAll-clustering-data-CCN/allAuthors_id2topicInfo_global_clusterNum/resolutionParam_{0}.json".format(str(resolution_param))
    with open(path_2,'r', encoding='utf-8-sig') as fr_2:
        allAuthors_id2topicInfo_global_dict = json.load(fr_2)
    # 从APS所有作者列表中去掉所有诺奖得主，作为非合作者的待选列表1
    temp_list_1 = [e for e in allAuthors_id2topicInfo_global_dict.keys() if e not in all_laursID_list]
    
    path_3 = dirPath_3 + "/1-paperAll-clustering-data-CCN/allLaurs_id2topicInfo/resolutionParam_{0}.json".format(str(resolution_param))
    with open(path_3,'r', encoding='utf-8-sig') as fr_3:
        allLaurs_id2topic_global_dict = json.load(fr_3)
    allLaurs_prizeTopic_list_2d = [e["laurPapers_topic_unique"] for e in allLaurs_id2topic_global_dict.values()]
    allLaurs_prizeTopic_list = _flatten(allLaurs_prizeTopic_list_2d)
    del allLaurs_prizeTopic_list_2d
    
    path_4 = dirPath_3 + "/1-paperAll-clustering-data-CCN/allAuthors_id2paperInfo_withGlobalCluster/allAuthors_id2yearRange_dict.json"
    with open(path_4, 'r', encoding='utf-8') as fr_4:
        allAuthors_id2yearRange_dict = json.load(fr_4)
 
    # groupD1
    allLaurs_groupD1_temp_dict = {}
    # groupD2
    allLaurs_groupD2_temp_dict = {}
    # 存储匹配时的信息
    allLaurs_matchingNoncollas_info_dict = {}
    # 遍历所有诺奖得主
    for laur_i, infoDict_i in allLaurs_id2groupAInfo_dict.items():
        prize_year_i = infoDict_i['prize_year']
        groupA_list_i = infoDict_i['groupA_list']
        groupA_list_filt_i = [e for e in groupA_list_i if ((prize_year_i>=allAuthors_id2yearRange_dict[e]['start_pubYear'])&(prize_year_i < allAuthors_id2yearRange_dict[e]['end_pubYear']))]
        if len(groupA_list_i) == 0:
            continue
            
        # （1）从temp_list_1中去掉该诺奖得主的合作者
        temp_list_2 = [e for e in temp_list_1 if e not in allLaurs_id2collas_allPapers_dict[laur_i]]
        laurPapers_topic_list_i = allLaurs_id2topic_global_dict[laur_i]["laurPapers_topic_unique"]
        # 获取除开该诺奖得主的获奖主题号之外的其他诺奖得主的获奖主题列表
        others_prizeTopic_list_i = [e for e in allLaurs_prizeTopic_list if e not in laurPapers_topic_list_i]
        
        # （2）如果该作者的发文主题包含除开该诺奖得主的获奖主题外的获奖主题，则去掉该作者
        temp_list_3 = [e for e in temp_list_2 if not del_author_underTopics_func(e, others_prizeTopic_list_i, allAuthors_id2topicInfo_global_dict)] 
        del temp_list_2
        
        # （3）非合作者的发文年份区间应该包括诺奖得主的获奖年份
        temp_list_4 = [e for e in temp_list_3 if ((prize_year_i>=allAuthors_id2yearRange_dict[e]['start_pubYear'])&(prize_year_i < allAuthors_id2yearRange_dict[e]['end_pubYear']))]
        del temp_list_3
        
        # （4）遍历非合作者待选列表
        groupD1_temp_list_i = []
        groupD2_temp_list_i = []
        groupA_pubYear_start_set = set([allAuthors_id2yearRange_dict[e]['start_pubYear'] for e in groupA_list_i])
        # 去掉列表中的0（0表示该作者没有发文信息，因此没有发文年份，因此最早发文年份为0）
        groupA_pubYear_start_list = [e for e in groupA_pubYear_start_set if e!=0]
        if bool(year_diff) & (len(groupA_pubYear_start_list)!=0):
            min_groupC_pubYear_start = min(groupA_pubYear_start_list) - year_diff
            max_groupC_pubYear_start = max(groupA_pubYear_start_list) + year_diff
        else:
            min_groupC_pubYear_start = 0
            max_groupC_pubYear_start = 2030
                
        # 遍历待选列表中的所有作者ID
        for author_j in temp_list_4:
            pubYear_start_j = allAuthors_id2yearRange_dict[author_j]['start_pubYear']
            topic_list_j = [int(e) for e in allAuthors_id2topicInfo_global_dict[author_j].keys()]    
            if (pubYear_start_j >= min_groupC_pubYear_start) & (pubYear_start_j <= max_groupC_pubYear_start):
                if set(topic_list_j) & set(laurPapers_topic_list_i):
                    pubYear_pTopic_list_j = [int(e["pub_year"]) for e in allAuthors_id2paperInfo_dict[author_j]["papers_info"].values() if e[cluster_param] in laurPapers_topic_list_i]
                    # groupD_1：在诺奖颁布前就已经发表了获奖主题论文
                    if min(pubYear_pTopic_list_j) <= prize_year_i:
                        groupD1_temp_list_i.append(author_j)
                    # groupD_2：在诺奖颁布后才开始发表获奖主题论文
                    else:
                        groupD2_temp_list_i.append(author_j)
          
        allLaurs_groupD1_temp_dict[laur_i] = groupD1_temp_list_i
        allLaurs_groupD2_temp_dict[laur_i] = groupD2_temp_list_i
        info = "Laureate {0} (prize year:{1}) roughly has {2} groupD1 and {3} groupD2 authors (groupA: {4}/ {5}).".format(laur_i, 
                                                                                                                        prize_year_i,
                                                                                                                      len(groupD1_temp_list_i),
                                                                                                                      len(groupD2_temp_list_i),
                                                                                                                      len(groupA_list_i),
                                                                                                                      len(groupA_list_filt_i)
                                                                                                                     )
        print(info)
        allLaurs_matchingNoncollas_info_dict[laur_i] = info
        del groupD1_temp_list_i
        del groupD2_temp_list_i
        
    print("= "*15 + "Writing to files... " +"= "*15)
    path_6 = dirPath_3 + "/7-nonCollas-groupD-tempList-matching/"
    path_7 = path_6 + "/temp_groupD_matchingInfo/resolutionParam_{0}/".format(resolution_param)
    df = pd.DataFrame.from_dict(allLaurs_matchingNoncollas_info_dict, orient='index', columns=['tempD_matchingInfo'])
    df = df.reset_index().rename(columns={'index':'laureate_id'})
    df.to_csv(path_7+"/year_diff_{0}.csv".format(year_diff), encoding='utf-8-sig', index=False)
    
    # 临时groupD1
    dict_to_json_func1(path_6 + "/allLaurs_groupD1_tempList/resolutionParam_{0}/".format(resolution_param), 
                                 "year_diff_{0}.json".format(year_diff),
                                 allLaurs_groupD1_temp_dict
                                )
    # 临时groupD2
    dict_to_json_func1(path_6 + "/allLaurs_groupD2_tempList/resolutionParam_{0}/".format(resolution_param), 
                                 "year_diff_{0}.json".format(year_diff),
                                 allLaurs_groupD2_temp_dict
                                )
    
    return allLaurs_matchingNoncollas_info_dict

In [17]:
get_groupD_tempList_func(resolution_param=0.1, 
                         year_diff=None, 
                         cluster_param='global_clusterNum'
                        )

Laureate 52053 (prize year:1927) roughly has 11 groupD1 and 6 groupD2 authors (groupA: 1/ 1).
Laureate 191584 (prize year:1934) roughly has 4 groupD1 and 0 groupD2 authors (groupA: 3/ 3).
Laureate 202601 (prize year:1944) roughly has 1 groupD1 and 2 groupD2 authors (groupA: 2/ 2).
Laureate 7060 (prize year:1955) roughly has 41 groupD1 and 13 groupD2 authors (groupA: 2/ 2).
Laureate 25747 (prize year:1956) roughly has 41 groupD1 and 12 groupD2 authors (groupA: 3/ 3).
Laureate 138251 (prize year:1963) roughly has 46 groupD1 and 14 groupD2 authors (groupA: 1/ 1).
Laureate 155444 (prize year:1963) roughly has 149 groupD1 and 3 groupD2 authors (groupA: 1/ 1).
Laureate 7733 (prize year:1964) roughly has 32 groupD1 and 10 groupD2 authors (groupA: 2/ 2).
Laureate 87585 (prize year:1965) roughly has 28 groupD1 and 9 groupD2 authors (groupA: 1/ 1).
Laureate 103834 (prize year:1967) roughly has 164 groupD1 and 37 groupD2 authors (groupA: 1/ 1).
Laureate 111984 (prize year:1968) roughly has 185 gr

{'52053': 'Laureate 52053 (prize year:1927) roughly has 11 groupD1 and 6 groupD2 authors (groupA: 1/ 1).',
 '191584': 'Laureate 191584 (prize year:1934) roughly has 4 groupD1 and 0 groupD2 authors (groupA: 3/ 3).',
 '202601': 'Laureate 202601 (prize year:1944) roughly has 1 groupD1 and 2 groupD2 authors (groupA: 2/ 2).',
 '7060': 'Laureate 7060 (prize year:1955) roughly has 41 groupD1 and 13 groupD2 authors (groupA: 2/ 2).',
 '25747': 'Laureate 25747 (prize year:1956) roughly has 41 groupD1 and 12 groupD2 authors (groupA: 3/ 3).',
 '138251': 'Laureate 138251 (prize year:1963) roughly has 46 groupD1 and 14 groupD2 authors (groupA: 1/ 1).',
 '155444': 'Laureate 155444 (prize year:1963) roughly has 149 groupD1 and 3 groupD2 authors (groupA: 1/ 1).',
 '7733': 'Laureate 7733 (prize year:1964) roughly has 32 groupD1 and 10 groupD2 authors (groupA: 2/ 2).',
 '87585': 'Laureate 87585 (prize year:1965) roughly has 28 groupD1 and 9 groupD2 authors (groupA: 1/ 1).',
 '103834': 'Laureate 103834 (p